In [8]:
import pandas as pd

In [9]:
data = pd.read_csv('./data/data_1.csv')

C:\Users\USER\AppData\Local\Temp\ipykernel_8900\13301075.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./data/data_1.csv')


# 1.

In [12]:
temp = data.isna().sum()[data.isna().sum()>0]
temp.max() - temp.min()

np.int64(460408)

# 2.

In [19]:
data = data.loc[:, data.isna().sum() < 500].dropna()
len(data)

460287

# 3.

In [24]:
data.상권업종대분류코드.value_counts() / len(data.상권업종대분류코드)

상권업종대분류코드
I2    0.273612
G2    0.214542
M1    0.187205
S2    0.101261
P1    0.056476
L1    0.050986
N1    0.046504
R1    0.039525
Q1    0.016692
I1    0.013198
Name: count, dtype: float64

# 4.

In [36]:
data.query('시군구명=="강남구"').상권업종소분류명.value_counts()

상권업종소분류명
경영 컨설팅업            5107
부동산 중개/대리업         3417
광고 대행업             3077
백반/한정식             2103
카페                 1926
                   ... 
분류 안된 외국식 음식점         2
그 외 기타 숙박업            2
비디오방                  1
음반/비디오물 대여업           1
바둑/장기/체스 경기 운영업       1
Name: count, Length: 245, dtype: int64

# 5.

In [38]:
data.query('시군구명=="종로구"').상권업종소분류명.value_counts()

상권업종소분류명
백반/한정식             1177
카페                 1111
시계/귀금속 소매업          648
여행사                 613
경영 컨설팅업             611
                   ... 
수상/해양 레저업             1
방사선 진단/병리 검사 의원       1
일반병원                  1
캠핑/글램핑                1
가축 사료 소매업             1
Name: count, Length: 244, dtype: int64

# 6.

In [47]:
data.query('상권업종소분류명=="편의점"').상호명.str.extract('(세븐일레븐|세븐|씨유|CU|GS|지에스).+', expand=False).value_counts()

상호명
씨유       2110
지에스      1710
세븐일레븐    1371
세븐       1149
GS       1096
CU        130
Name: count, dtype: int64

# 7.

In [55]:
(len(data.query('상권업종소분류명=="편의점"')) - data.query('상권업종소분류명=="편의점"').상호명.str.extract('(세븐일레븐|씨유|CU|GS|지에스).+', expand=False).value_counts().sum()) / len(data.query('상권업종소분류명=="편의점"'))

np.float64(0.3192279138827023)

In [54]:
(len(data.query('상권업종소분류명=="편의점"')) - data.query('상권업종소분류명=="편의점"').상호명.str.extract('(세븐일레븐|세븐|씨유|CU|GS|지에스).+', expand=False).value_counts().sum()) / len(data.query('상권업종소분류명=="편의점"'))

np.float64(0.19758192809417754)

# 8.

In [67]:
len(
    data.filter(items=['시군구명', '상호명', '위도', '경도'])
    .merge(data.filter(items=['시군구명', '위도', '경도']).groupby(['시군구명']).mean().rename(columns={'위도': '위도평균', '경도': '경도평균'}).reset_index(), on=['시군구명'])
    .query('위도 > 위도평균 and 경도 > 경도평균')
)

95126

# 9.

In [85]:
temp = data.filter(items=['시군구명', '상호명'])
temp['상호길이'] = data.상호명.str.len()
temp['target_len'] = temp.상호길이 >= 10
temp.filter(items=['시군구명', 'target_len']).groupby(['시군구명']).mean().sort_values(by=['target_len'], ascending=False).head(2)

,target_len
시군구명,
서초구,0.167329
송파구,0.149883


# 10.

In [92]:
from sklearn.metrics.pairwise import pairwise_distances
import numpy as np

In [119]:
average_coordinates = data.filter(items=['시군구명', '위도', '경도']).groupby(['시군구명']).mean()
distances = pairwise_distances(average_coordinates)
target_index = np.where(np.abs(distances.max() - distances) < 1e-10)[0]
average_coordinates.index[target_index]

Index(['강동구', '강서구'], dtype='object', name='시군구명')

# 11.

corr 그림을 그려보면 정상 데이터는 corr이 1에 가까운 것을 확인할 수 있음. <br>
corr이 직선을 벗어나는 점을 이상치로 간주하여 이상탐지 가능.